## Librerias

In [1]:
# Data
import pandas as pd
import numpy as np
import csv
import sqlite3

# Scraping
import requests
from bs4 import BeautifulSoup as bs
from requests_html import HTMLSession
import concurrent.futures
import time

# Otros
import random
from datetime import date
from tqdm.notebook import tqdm_notebook

## Funciones

In [2]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("à", "a"),
        ("è", "e"),
        ("ì", "i"),
        ("ò", "o"),
        ("ù", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [ ]:
'''def check_proxys():
    proxylist = []
    valid_proxys = []

    with open('data/proxylist.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            proxylist.append(row[0])

    def extract(proxy):
        try:
            r = requests.get('https://httpbin.org/ip', proxies={'http': proxy, 'https': proxy}, timeout=3)
            #print(r.json(),' - working')
            valid_proxys.append(proxy)
        except:
            #print(f'{proxy} - failed')
            pass
        return proxy

    with concurrent.futures.ThreadPoolExecutor() as exector:
        exector.map(extract, proxylist)
        
    return valid_proxys'''

In [3]:
def user_agents_list():
    
    user_agents = []
    
    with open('data/user_agents.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            user_agents.append(row[0])

    return user_agents

In [4]:
def lista_zonas():
    
    user_agents = user_agents_list()
    user = user_agents[random.randrange(0,len(user_agents))]
    
    r = session.request(method='GET',url="https://www.pisos.com/pisos/",headers={'user-agent':user})#,proxies={'http': proxy, 'https': proxy})
    
    return [e.split('-')[1][:-1] for e in r.html.links if '/venta/'in e]

In [5]:
def busqueda_carac(termino,lista_carac):    
    
    res = ''
    
    i=0
    encontrado = False
    while i < len(lista_carac) and encontrado == False:
        if termino in lista_carac[i]:
            encontrado = True
            if len(lista_carac[i].split(':'))>1:
                res = lista_carac[i].split(':')[1].strip()
        else:
            i+=1
            
    return res

In [6]:
# Gestión de 1 sóla vivienda
def one_house(url):
    
    user = user_agents[random.randrange(0,len(user_agents))]
    
    try:
        r = session.request(method='GET',url=url, headers={'user-agent':user},timeout=(10))#,proxies={'http': proxy, 'https': proxy})
    
    except:
        pass
    
    else:
        soup = bs(r.text, 'html.parser')

        if soup.find('p', class_="notification__disabled--title") == None and soup.find('div', class_="no-results")== None:
            
            if soup.find_all('li',class_='charblock-element more-padding') != None:
                lista_carac = soup.find_all('li',class_='charblock-element more-padding')
                lista_carac = [e.text.replace('\n','').strip() for e in lista_carac]
            else:
                lista_carac = []
                
            if soup.find_all('li',class_='charblock-element element-with-bullet certificate-clasification') != None:
                certificado = soup.find_all('li',class_='charblock-element element-with-bullet certificate-clasification')
                certificado = [e.text.replace('\n','').strip() for e in certificado]
            else:
                certificado = []
                
            if soup.find('h1') != None:
                titulo = soup.find('h1').text
            else:
                titulo = ''
                
            if soup.find('h2') != None:
                zona = soup.find('h2').text
            else:
                zona = ''
                
            if soup.find('span',class_='h1 jsPrecioH1') != None:
                precio = soup.find('span',class_='h1 jsPrecioH1').text[:-2].replace('.','')
            else:
                precio = ''
                
            if soup.find(id='descriptionBody') != None:
                descr = soup.find(id='descriptionBody').text.split('\n')[1].strip()
            else:
                descr = ''
            
            if soup.find('div', class_="updated-date") != None:
                f_entrada = soup.find('div', class_="updated-date").text.strip().split(' ')[2]
            else:
                f_entrada = f"{date.today().day}/{date.today().month}/{date.today().year}"
                
            vivienda = {
                'id':url[-19:-1],
                'titulo':titulo, 
                'zona':zona, 
                'precio':precio, 
                'descripcion': descr,
                'superficie_construida_m': busqueda_carac('Superficie construida',lista_carac).replace('.','')[:-3],
                'superficie_util_m': busqueda_carac('útil',lista_carac).replace('.','')[:-3],
                'superficie_solar_m': busqueda_carac('solar',lista_carac).replace('.','')[:-3],
                'habitaciones': busqueda_carac('Habitaciones',lista_carac),
                'banyos': busqueda_carac('Baños',lista_carac),
                'planta': busqueda_carac('Planta',lista_carac)[:-1],
                'antiguedad': busqueda_carac('Antigüedad',lista_carac),
                'gastos_comunidad': busqueda_carac('comunidad',lista_carac),
                'conservacion': busqueda_carac('Conservación',lista_carac),
                'f_entrada': f_entrada,
                'url':url
            }

            #
            # IMPORTANTE . Queda pendiente incluir el eur_m en el dataframe
            #
            #if vivienda['superficie_construida_m'] != '':
            #    vivienda['eur_m']= int(vivienda['precio'])/ int(vivienda['superficie_construida_m'])
            #else:
            #    vivienda['eur_m']= ''

            time.sleep(random.randint(0,2))

            return vivienda
        else:
            #print('Anuncio caido', url)
            pass

In [15]:
def check_alive(link):
    user = user_agents[random.randrange(0,len(user_agents))]
    
    try:
        r = session.request(method='GET',url=link, headers={'user-agent':user},timeout=(10))#,proxies={'http': proxy, 'https': proxy})
    
    except:
        pass
    
    else:
        soup = bs(r.text, 'html.parser')

        if soup.find('p', class_="notification__disabled--title") != None or soup.find('div', class_="no-results")!= None:
            return link
        

## Main

In [8]:
# Inicializar componentes
session = HTMLSession()

#valid_proxys = check_proxys()
user_agents = user_agents_list()
zonas = lista_zonas()

In [ ]:
%%time

# Obtener TODOS los enlaces de la web por zonas

user = user_agents[random.randrange(0,len(user_agents))]
#proxy = valid_proxys[random.randrange(0,len(valid_proxys))]
links_por_zonas = {}

for zona in zonas:
    
    # Primera página
    i = 1
    continuar = True
    r = session.request(method='GET',url=f"https://www.pisos.com/venta/pisos-{zona}/fecharecientedesde-desc/",headers={'user-agent':user})#,proxies={'http': proxy, 'https': proxy})
    links = [e for e in r.html.absolute_links if '/comprar/'in e]
    check_duplicates = links
    
    # Resto de páginas 
    while r.status_code == 200 and i < 101 and continuar == True: 
        time.sleep(random.randint(0,2)) # Modificado de 5 a 2 para paralelizar
        i+=1
        user = user_agents[random.randrange(0,len(user_agents))]
        #proxy = valid_proxys[random.randrange(0,len(valid_proxys))]

        try:
            r = session.request(method='GET',url=f"https://www.pisos.com/venta/pisos-{zona}/fecharecientedesde-desc/{i}/", headers={'user-agent':user})#,proxies={'http': proxy, 'https': proxy})

        except:
            print('Nos han pillado. Hemos llegado a la página',i, 'Zona:', zona)

        else:
            if r.status_code == 200:
                
                links_aux = [e for e in r.html.absolute_links if '/comprar/'in e]
                
                if check_duplicates != links_aux:
                    
                    check_duplicates = links_aux
                    for link in links_aux:
                        links.append(link)
                else:
                    continuar = False
                    
    links_por_zonas[zona] = links

time.sleep(300)

In [ ]:
# Muestra links obtenidos y elimina duplicados por comunidad

for key in links_por_zonas:
    print('Zona', key)
    print('Enlaces obtenidos:',len(links_por_zonas[key]))
    links_por_zonas[key] = list(set(links_por_zonas[key]))
    print('Enlaces únicos:',len(links_por_zonas[key]))

In [ ]:
# Guardado de enlaces

with open('data/links.csv','w') as file:
    for key in links_por_zonas:
        file.write(key)
        file.write('\n')
        for link in links_por_zonas[key]:
            file.write(link)
            file.write('\n')

In [ ]:
# Carga de enlaces del csv

links_por_zonas=[]

with open('data/links.csv', 'r') as fichero:
    for linea in fichero.readlines():
        if 'http' in linea:
            links_por_zonas.append(linea.strip())
len(links_por_zonas)

In [ ]:
# Eliminamos enlaces comunes entre comunidades

links_por_zonas = list(set(links_por_zonas))
len(links_por_zonas)

In [ ]:
# Consulta a la BBDD para saber si los enlaces ya están guardados

con = sqlite3.connect('pisos.db')
cur = con.cursor()
cur.execute('''SELECT url FROM pisos''')
links_bbdd = cur.fetchall()
links_bbdd = [e[0] for e in links_bbdd]
con.close()

In [ ]:
%%time

links_por_zonas = [e for e in links_por_zonas if e not in links_bbdd]

In [ ]:
len(links_por_zonas)

In [ ]:
## Ejecutar aqui

In [ ]:
%%time

data = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for link in links_por_zonas:
        futures.append(executor.submit(one_house, link))
    for future in concurrent.futures.as_completed(futures):
        #print(future.result())
        data.append(future.result())

In [ ]:
# Enlaces fallidos

i = 0
for e in data:
    if e == None:
        i+=1
i

## Limpiar la data

In [ ]:
data_no_nulos = [e for e in data if e != None]

In [ ]:
len(data_no_nulos)

In [ ]:
# Creación de dataframe
df = pd.DataFrame(data_no_nulos)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#Cambio de formato a columnas a numero
num_cols = df.columns.drop(['id', 'zona','titulo', 'descripcion', 'habitaciones', 'banyos', 'planta', 'antiguedad', 'gastos_comunidad', 'conservacion', 'url', 'f_entrada'])
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce') 


# Reemplazo por nulos valores vacios
df.habitaciones = df.habitaciones.replace('', np.nan) 
df.banyos = df.banyos.replace('', np.nan)
df.antiguedad = df.antiguedad.replace('', None)
df.gastos_comunidad = df.gastos_comunidad.replace('', None)
df.conservacion = df.conservacion.replace('', None)


# Nuevas columna eur_m
df['eur_m'] = df.precio/df.superficie_construida_m


#Limpieza de valores no deseados
df = df[~df.eur_m.isna()].reset_index(drop=True) #Eliminación de viviendas sin precio o sin metros cuadrados
df = df[(df.superficie_construida_m>20)&(df.superficie_construida_m<800)].reset_index(drop=True) #Eliminación de viviendas que tengan menos de 20 metros construidos y mas de 800

#Eliminación de fincas y estudios
bad_indexs = []
for ind, e in enumerate(df.descripcion): 
    if 'estudio' in e:
        bad_indexs.append(ind)
for ind, e in enumerate(df.descripcion[(df.superficie_solar_m==df.superficie_construida_m)&(df.habitaciones.isna())]): 
    if 'finca' in e:
        bad_indexs.append(ind)

df = df.drop(bad_indexs).reset_index(drop=True) 


# Nueva columna municipio
df['municipio'] = [e.split('(')[1].split(')')[0] if '(' in e else e for e in df.zona]

#Normalización de formatos y gestion de nulos
df.zona = [e.split('(')[0] for e in df.zona]
df.municipio = [e.split('.')[1].strip() if '.' in e else e for e in df.municipio]
df.municipio = [e.replace('Capital','').strip() for e in df.municipio]
df.municipio = df.municipio.apply(normalize)

#Incluimos CPs de municipios
df.municipio = [e.upper() for e in df.municipio]
codigos_p = pd.read_csv('data/cp_data.csv',sep=';')
codigos_p.zona = [e.split('(')[0].strip() for e in codigos_p.zona]
dict_com_cp = dict(zip(codigos_p.zona, codigos_p.cp))
df['codigo_postal'] = df.municipio.map(dict_com_cp)

#Asignamos comunidades con CPs de municipios
df['comunidad'] = [str(e)[2:4] for e in df.codigo_postal]
leyenda_cp = pd.read_csv('data/leyenda_cp.csv',sep=';')
dict_leyenda = dict(zip([e[:2] for e in leyenda_cp.cod], leyenda_cp.comunidad)) #Habria que alimentar el csv para los null
df.comunidad = df.comunidad.map(dict_leyenda)

# Por el momento reventamos los 3000 nulos de comunidad
bad_indexs = df[df.comunidad.isna()].index.values
df = df.drop(bad_indexs).reset_index(drop=True) 

df.comunidad = df.comunidad.apply(normalize) #Ahora puedo normalizar
df['active']=1
df['tmstmp']=f"{date.today().day}/{date.today().month}/{date.today().year}"

In [ ]:
df.shape

In [ ]:
# Reordenación de columnas
df = df[['id', 'titulo', 'zona','municipio', 'comunidad', 'codigo_postal','precio', 'eur_m', 'descripcion',
       'superficie_construida_m', 'superficie_util_m', 'superficie_solar_m',
       'habitaciones', 'banyos', 'planta', 'antiguedad', 'gastos_comunidad',
       'conservacion', 'url', 'f_entrada','active', 'tmstmp']]

## Database

In [10]:
# Inicialización de la conexión con la BBDD

con = sqlite3.connect('pisos.db', check_same_thread=False)
cur = con.cursor()

In [ ]:
#Inicialización de la tabla

df = df.T
list_to_db = [df[i] for i in range (0, df.shape[1])]

cur.execute('''CREATE TABLE IF NOT EXISTS pisos
                    (id text PRIMARY KEY, 
                    titulo text,
                    zona text,
                    municipio text,
                    comunidad text,
                    codigo_postal text,
                    precio real, 
                    eur_m real,
                    descripcion text, 
                    superficie_construida_m real, 
                    superficie_util_m real,
                    superficie_solar_m real,
                    habitaciones int,
                    banyos int,
                    planta text,
                    antiguedad text,
                    gastos_comunidad text,
                    conservacion text,
                    url text,
                    f_entrada text,
                    active bool,
                    tmstmp text
                    )
            ''')


# Inserción de valores

cur.executemany("INSERT OR IGNORE INTO pisos VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",list_to_db)
con.commit()

df = df.T #Restablecemos el df

In [ ]:
%%time
# Actualizamos los valores de alive y tmstmp

cur.execute('''SELECT url FROM pisos WHERE active = 1''')
links_bbdd = cur.fetchall()
links_bbdd = [e[0] for e in links_bbdd]

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for link in links_bbdd:
        aux = executor.submit(check_alive, link)
        if aux != None:
            futures.append(aux)

for link_ko in futures:
    cur.execute(f'''SELECT id FROM pisos WHERE url = "{link}"''')
    id_bbdd = cur.fetchone()            
    cur.execute('''UPDATE pisos SET active = (?) WHERE id = (?)''', ('0', id_bbdd[0]) )
    cur.execute('''UPDATE pisos SET tmstmp = (?) WHERE id = (?)''', (f'{date.today().day}/{date.today().month}/{date.today().year}', id_bbdd[0]) )
    
con.commit()

In [ ]:
#EJECUTAR SIEMPRE ANTES DE CERRAR
con.close()

In [ ]:
# Funcion para actualizar cualquier columna (tmstmp ahora)

#for i in range(len(df.id)):
#    cur.execute('''UPDATE pisos SET tmstmp = (?) WHERE id = (?)''', (df.tmstmp[i], df.id[i]) )
#con.commit()

In [ ]:
#cur.execute('''SELECT id FROM pisos WHERE url = "https://www.pisos.com/comprar/duplex-huelves_centro_urbano-945856909855543_109800/"''')
#cur.fetchone()

In [ ]:
# Plantilla para eliminar registros

#con = sqlite3.connect('pisos.db')
#cur = con.cursor()

#for e in bad_index:
#    cur.execute(f'''DELETE FROM pisos WHERE id = "{e}"''')
#con.commit()

#con.close()

In [ ]:
#links_por_zonas[:5]